In [1]:
import os
import glob
from PIL import Image
import pandas as pd
import cv2
from IPython.display import display


def get_image_info(folder_path):
    image_data = []

    for file in glob.glob(os.path.join(folder_path, "*")):
        try:
            with Image.open(file) as img:
                img_resized = img.resize((50, 50))

                file_info = {
                    "Ім'я файлу": os.path.basename(file),
                    "Формат": img.format,
                    "Кольорова модель": img.mode,
                    "Кількість каналів": len(img.getbands()),
                    "Розмір файлу (MB)": round(os.path.getsize(file) / (1024 * 1024), 2),
                    "Ширина": img.width,
                    "Висота": img.height,
                    "Повний шлях": os.path.abspath(file),
                    "Зменшене зображення": img_resized
                }
                image_data.append(file_info)

        except (Image.UnidentifiedImageError, OSError):
            try:
                img_cv = cv2.imread(file)
                if img_cv is not None:
                    height, width, channels = img_cv.shape
                    file_info = {
                        "Ім'я файлу": os.path.basename(file),
                        "Формат": "AVIF/Unknown",
                        "Кольорова модель": f"{channels}-канальна",
                        "Кількість каналів": channels,
                        "Розмір файлу (MB)": round(os.path.getsize(file) / (1024 * 1024), 2),
                        "Ширина": width,
                        "Висота": height,
                        "Повний шлях": os.path.abspath(file),
                        "Зменшене зображення": cv2.resize(img_cv, (50, 50))
                    }
                    image_data.append(file_info)
            except Exception as e:
                print(f"Файл '{file}' пропущено: {e}")

    df = pd.DataFrame(image_data)
    return df


if __name__ == "__main__":
    folder_path = "images_folder"
    if os.path.exists(folder_path):
        image_df = get_image_info(folder_path)
        display(image_df)
    else:
        print(f"Папка '{folder_path}' не існує. Перевірте шлях.")

,Ім'я файлу,Формат,Кольорова модель,Кількість каналів,Розмір файлу (MB),Ширина,Висота,Повний шлях,Зменшене зображення
0,59cfb84763df615ed367d6f1.png,PNG,RGB,3,0.07,800,480,D:\Python\Solventus\lab9\.venv\Scripts\images_...,<PIL.Image.Image image mode=RGB size=50x50 at ...
1,download (2).jpg,JPEG,RGB,3,0.01,204,192,D:\Python\Solventus\lab9\.venv\Scripts\images_...,<PIL.Image.Image image mode=RGB size=50x50 at ...
2,download (3).jpg,JPEG,RGB,3,0.01,182,172,D:\Python\Solventus\lab9\.venv\Scripts\images_...,<PIL.Image.Image image mode=RGB size=50x50 at ...
3,download (4).jpg,JPEG,RGB,3,0.01,278,181,D:\Python\Solventus\lab9\.venv\Scripts\images_...,<PIL.Image.Image image mode=RGB size=50x50 at ...
4,download.jpg,JPEG,RGB,3,0.01,204,192,D:\Python\Solventus\lab9\.venv\Scripts\images_...,<PIL.Image.Image image mode=RGB size=50x50 at ...
5,dwoilp3BVjlE.webp,WEBP,RGB,3,0.41,1920,1080,D:\Python\Solventus\lab9\.venv\Scripts\images_...,<PIL.Image.Image image mode=RGB size=50x50 at ...
6,images (1).jpg,JPEG,RGB,3,0.01,201,251,D:\Python\Solventus\lab9\.venv\Scripts\images_...,<PIL.Image.Image image mode=RGB size=50x50 at ...
7,images (2).jpg,JPEG,RGB,3,0.01,275,183,D:\Python\Solventus\lab9\.venv\Scripts\images_...,<PIL.Image.Image image mode=RGB size=50x50 at ...
8,images (3).jpg,JPEG,RGB,3,0.01,225,225,D:\Python\Solventus\lab9\.venv\Scripts\images_...,<PIL.Image.Image image mode=RGB size=50x50 at ...
9,images (4).jpg,JPEG,RGB,3,0.00,299,168,D:\Python\Solventus\lab9\.venv\Scripts\images_...,<PIL.Image.Image image mode=RGB size=50x50 at ...


In [2]:
import os
import random
import glob
from PIL import Image, ImageEnhance

def crop_to_square(img, target_size=500):
    width, height = img.size
    side_length = min(width, height)
    left = (width - side_length) // 2
    top = (height - side_length) // 2
    right = left + side_length
    bottom = top + side_length
    img_cropped = img.crop((left, top, right, bottom))
    return img_cropped.resize((target_size, target_size), Image.Resampling.LANCZOS)


def apply_random_filter(img):
    img = img.convert("RGB")
    filters = ['red', 'green', 'blue', 'yellow', 'cyan', 'magenta']
    selected_filter = random.choice(filters)
    r, g, b = img.split()

    if selected_filter == 'red':
        r = r.point(lambda p: min(p + 50, 255))
    elif selected_filter == 'green':
        g = g.point(lambda p: min(p + 50, 255))
    elif selected_filter == 'blue':
        b = b.point(lambda p: min(p + 50, 255))
    elif selected_filter == 'yellow':
        r = r.point(lambda p: min(p + 30, 255))
        g = g.point(lambda p: min(p + 30, 255))
    elif selected_filter == 'cyan':
        g = g.point(lambda p: min(p + 30, 255))
        b = b.point(lambda p: min(p + 30, 255))
    elif selected_filter == 'magenta':
        r = r.point(lambda p: min(p + 30, 255))
        b = b.point(lambda p: min(p + 30, 255))

    return Image.merge("RGB", (r, g, b))


def create_poster(folder_path, output_image, cols, rows, image_size=500):
    all_images = glob.glob(os.path.join(folder_path, "*"))
    images = [file for file in all_images if file.lower().endswith(('png', 'jpg', 'jpeg', 'webp'))]

    if len(images) == 0:
        raise ValueError("У папці немає підтримуваних зображень!")

    poster_width = cols * image_size
    poster_height = rows * image_size
    poster = Image.new("RGB", (poster_width, poster_height), "white")

    for row in range(rows):
        for col in range(cols):
            try:
                img_path = random.choice(images)
                with Image.open(img_path) as img:
                    img_square = crop_to_square(img, image_size)
                    img_filtered = apply_random_filter(img_square)

                    x = col * image_size
                    y = row * image_size
                    poster.paste(img_filtered, (x, y))
            except Exception as e:
                print(f"Помилка при обробці {img_path}: {e}")

    poster.save(output_image, quality=95)
    print(f"Постер збережено як {output_image}")


if __name__ == "__main__":
    folder_path = "images_folder"
    os.makedirs("output_posters", exist_ok=True)

    create_poster(folder_path, "output_posters/poster1.jpg", cols=3, rows=2)
    create_poster(folder_path, "output_posters/poster2.jpg", cols=4, rows=3)
    create_poster(folder_path, "output_posters/poster3.jpg", cols=5, rows=4)

    print("Створено 3 постери!")


Постер збережено як output_posters/poster1.jpg
Постер збережено як output_posters/poster2.jpg
Постер збережено як output_posters/poster3.jpg
Створено 3 постери!
